# Setup

In [53]:
%pip install openai tiktoken

Note: you may need to restart the kernel to use updated packages.


In [54]:
from openai import OpenAI
client = OpenAI()
llm = "gpt-4-1106-preview"

In [55]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        # print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        # print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


def num_tokens_from_string(string, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a string."""
    return num_tokens_from_messages([{"content": string}], model=model)


# Load the Assistant

In [56]:
from datetime import datetime


# basic instructions
system_prompt = """
You are a Talent Acquisition and Development Specialist. 
Your task is to do a job analysis and identify the knowledge, skills, abilities, and other characteristics (KSAOs) as competencies for a hiring team to assess in a candidate. 
Lower level jobs should have less competencies and higher level jobs should have more, but you should cover all the requirements in the job desccription. 
You should also include competencies or word competencies that are aligned with the company values and if a competency is aligned with a company value, you should say so.
Refer to the actual company name rather than 'organization' or 'company'.
Don't use the word 'technological' instead use 'technology' or 'technical'.
Cite and or quote the job description as appropriate.
Include the job description verbatim, but change the heading levels to be just bolded text.
Be sure to cover specific technologies, systems or skills mentioned in the job description in the competencies or competency detail.
"""

# system_prompt += "\n\nFor this test, only produce 1 competencies."

# add today's date and time to include in the document
system_prompt += "\nToday's date and time is " + datetime.now().strftime("%B %d, %Y %-I:%M %p") + ".\n"

# instructions for processing the document template. We will reuse this.
doc_prompt = """
You will be provided below with a template in Jinja2 syntax delimited by triple quotes, and your task is to fully populate the template.
Always complete all loops fully. 
Always complete the metadata 
Always include a blank line after a markdown block and before a list
Do NOT ommit or truncate any content or sections. 
Do NOT ommit blank spaces or lines.
Do NOT include any delimiters or commentary in the output.
Before writing each section think about what you will write and include your thinking in HTML comments
When you are completely finished with the entire document write STOPSTOP.
"""
system_prompt += doc_prompt

# add the document template
with open('templates\prompt_template.md', 'r') as file:
    system_prompt += "\n\"\"\"\n" + file.read() + "\n\"\"\"\n\n"
    
print(system_prompt)


You are a Talent Acquisition and Development Specialist. 
Your task is to do a job analysis and identify the knowledge, skills, abilities, and other characteristics (KSAOs) as competencies for a hiring team to assess in a candidate. 
Lower level jobs should have less competencies and higher level jobs should have more, but you should cover all the requirements in the job desccription. 
You should also include competencies or word competencies that are aligned with the company values and if a competency is aligned with a company value, you should say so.
Refer to the actual company name rather than 'organization' or 'company'.
Don't use the word 'technological' instead use 'technology' or 'technical'.
Cite and or quote the job description as appropriate.
Include the job description verbatim, but change the heading levels to be just bolded text.
Be sure to cover specific technologies, systems or skills mentioned in the job description in the competencies or competency detail.

Today's d

## Job Description

In [57]:
job_description = ""
with open("examples/product-designer/job-description.md", "r") as file:
    job_description = file.read()

company_description = ""
with open("examples/product-designer/about-company.md", "r") as file:
    company_description = file.read()

user_prompt = "I am supplying you with a job description and a company description.\n\n"
user_prompt += "Job description is as follows:\n\n\"\"\"\n" + job_description + "\n\"\"\"\n\n"
user_prompt += "Company description is as follows:\n\n\"\"\"\n" + company_description + "\n\"\"\"\n\n"
user_prompt += "Proceed generating the full document. Do not truncate or abridge the document. Follow the template exactly.\n\n"

print(user_prompt)


I am supplying you with a job description and a company description.

Job description is as follows:

"""
## Job Title: Product Designer

### About Tiny

If you've ever written content on the web, chances are you've used a Tiny product. As part of the Tiugo Technologies portfolio of brands, we make TinyMCE and other great developer tools. Our team is dedicated to bringing content creation tools globally through open-source initiatives.

We're expanding our product family and are looking for a talented Product Designer to join our remote team of over 200 members worldwide, with significant presence in Australia, Poland, and the USA.

### About the Role

As a Product Designer for TinyMCE, you will have a direct impact on millions of users globally. In the TinyMCE product team, you will be part of the entire development journey, from identifying issues to defining and designing solutions. Your responsibilities include creating wireframes, prototypes, UI designs, and considering the broade

## Product the Draft

In [58]:
import os

def generate_output(system_prompt, user_prompt, llm):
    STOP_SIGNAL = "STOPSTOP"
    output = "" 

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    input_tokens = 0
    output_tokens = 0

    while True:
        try:
            stream = client.chat.completions.create(
                model=llm,
                max_tokens=4096,
                temperature=0,
                messages=messages,
                stream=True,
            )
            input_tokens += num_tokens_from_messages(messages, model=llm)

            new_output = ""

            for chunk in stream:
                if chunk.choices[0].delta.content is not None:
                    print(chunk.choices[0].delta.content, end="")
                    new_chunk = chunk.choices[0].delta.content
                    new_output += new_chunk

                if STOP_SIGNAL in new_output:
                    break

            output_tokens += num_tokens_from_string(new_output, model=llm)       
            output += new_output
            if not new_output or STOP_SIGNAL in new_output:
                break

            messages.append({"role": "assistant", "content": new_output})
        except Exception as e:
            print("Error:", e)
            # Handle the error here, such as retrying or terminating the loop
    
    return output, input_tokens, output_tokens


output, input_tokens, output_tokens = generate_output(system_prompt, user_prompt, llm)

---
title: Interview Guide for Product Designer
subtitle: Tiny
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 19, 2024 2:10 PM
--- 

# Interview Guide for Product Designer

## About this Guide

This guide is designed to assist the hiring team at Tiny in identifying and evaluating the best candidates for the Product Designer role, specifically for the TinyMCE product. It provides a structured approach to the interview process, ensuring that each candidate is assessed fairly and consistently across a range of competencies that are critical for success in the role.

The guide includes a detailed job description, an interview agenda, a competency framework aligned with Tiny's values and requirements, and a set of questions and scenarios to probe each competency. Additionally, it offers a case study for role play and a candidate Q&A section to further gauge fit and expertise.

By following this guide, interviewers will be able to make informed dec

In [59]:
# give update on the cost

cost = (input_tokens / 1000 * 0.01) + (output_tokens / 1000 * 0.03)
rounded_cost = round(cost, 2)
print("Total cost is approximately $" + str(rounded_cost) + " USD.")

Total cost is approximately $0.26 USD.


## Produce the Final

In [60]:
draft_markdown = output.replace("STOPSTOP", "")

# Define the file path
file_path = os.path.join("temp_files", "draft.md")

# Save the final_markdown to the file
with open(file_path, "w") as file:
    file.write(draft_markdown)

# Print the file path
print("Draft markdown saved to:", file_path)

Draft markdown saved to: temp_files/draft.md


In [61]:
user_prompt = f"""
You are a Talent Acquisition and Development Specialist. Your task is to review a hiring guide that is in draft. 
Please review and improve it as necessary. 
In particular, review the quality and accuracy of the competencies and the interview questions.
Do NOT modify the job description as this has been provided to you by the hiring manager.
Cite and or quote the job description as appropriate.
If you think the list of competencies is good, you can leave it as is.
If you think the list of competencies is bad, you can delete it and write a new one.
If you think the list of competencies is good but needs improvement, you can edit it.
If you think the list of competencies is good but needs more competencies, you can add more.
If you think the list of competencies is good but needs less competencies, you can delete some.
If you think the list of competencies is good but needs to be re-ordered, you can re-order them.
If a description is aligned with a company value, you should say so.
Expand the 'About this Guide' section to at least 3 paragraphs.
Be sure to cover specific technologies, systems or skills mentioned in the job description in the competencies or competency detail.
Follow the template from your system instructions.

Job description is as follows:
\"\"\"
{job_description}
\"\"\"

Company description is as follows:
\"\"\"
{company_description}
\"\"\"

Draft interview guide is as follows:
\"\"\"
{draft_markdown}
\"\"\"

"""
output, input_tokens, output_tokens = generate_output(system_prompt, user_prompt, llm)
final_markdown = output.replace("STOPSTOP", "")

# Define the file path
file_path = os.path.join("temp_files", "final.md")

# Save the final_markdown to the file
with open(file_path, "w") as file:
    file.write(final_markdown)

# Print the file path
print("Final markdown saved to:", file_path)

---
title: Interview Guide for Product Designer
subtitle: Tiny
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 19, 2024 2:10 PM
--- 

# Interview Guide for Product Designer

## About this Guide

This guide is crafted to assist the hiring team at Tiny in the meticulous process of selecting the ideal candidate for the Product Designer role, with a particular focus on the TinyMCE product. It is structured to provide a comprehensive and consistent assessment framework, ensuring that each candidate is evaluated fairly across a range of competencies that are pivotal for the role's success.

The guide encompasses a detailed job description, a structured interview agenda, a competency framework that aligns with Tiny's values and role requirements, and a series of questions and situational scenarios designed to probe each competency. Additionally, it includes a case study for role play and a candidate Q&A section to further assess fit and expertise.

B

In [62]:
# update on the cost
cost = (input_tokens / 1000 * 0.01) + (output_tokens / 1000 * 0.03)
rounded_cost = round(cost, 2)
print("Total cost is approximately $" + str(rounded_cost) + " USD.")

Total cost is approximately $0.38 USD.


## Diff

In [63]:
import difflib

# Split the markdown strings into lines
draft_lines = draft_markdown.splitlines()
final_lines = final_markdown.splitlines()

# Compute the diff between the two sets of lines
diff = difflib.unified_diff(draft_lines, final_lines)

# Print the diff
for line in diff:
    print(line)


--- 

+++ 

@@ -9,11 +9,11 @@

 
 ## About this Guide
 
-This guide is designed to assist the hiring team at Tiny in identifying and evaluating the best candidates for the Product Designer role, specifically for the TinyMCE product. It provides a structured approach to the interview process, ensuring that each candidate is assessed fairly and consistently across a range of competencies that are critical for success in the role.
-
-The guide includes a detailed job description, an interview agenda, a competency framework aligned with Tiny's values and requirements, and a set of questions and scenarios to probe each competency. Additionally, it offers a case study for role play and a candidate Q&A section to further gauge fit and expertise.
-
-By following this guide, interviewers will be able to make informed decisions based on a comprehensive understanding of each candidate's knowledge, skills, abilities, and other characteristics (KSAOs). This will help Tiny continue to build a team t

# Final Output

In [64]:
# Convert from markdown to docx
!pandoc -s temp_files/draft.md -o temp_files/draft.docx --reference-doc="templates/custom-reference.docx" --toc=True --toc-depth=1 --shift-heading-level-by -1

In [65]:
# Convert from markdown to docx
!pandoc -s temp_files/final.md -o temp_files/final.docx --reference-doc="templates/custom-reference.docx" --toc=True --toc-depth=1 --shift-heading-level-by -1

In [66]:

# display the final_markdown in the notebook
from IPython.display import Markdown
Markdown(final_markdown)

---
title: Interview Guide for Product Designer
subtitle: Tiny
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 19, 2024 2:10 PM
--- 

# Interview Guide for Product Designer

## About this Guide

This guide is crafted to assist the hiring team at Tiny in the meticulous process of selecting the ideal candidate for the Product Designer role, with a particular focus on the TinyMCE product. It is structured to provide a comprehensive and consistent assessment framework, ensuring that each candidate is evaluated fairly across a range of competencies that are pivotal for the role's success.

The guide encompasses a detailed job description, a structured interview agenda, a competency framework that aligns with Tiny's values and role requirements, and a series of questions and situational scenarios designed to probe each competency. Additionally, it includes a case study for role play and a candidate Q&A section to further assess fit and expertise.

By adhering to this guide, interviewers at Tiny will be equipped to make informed decisions based on a holistic understanding of each candidate's knowledge, skills, abilities, and other characteristics (KSAOs). This will enable Tiny to continue fostering a team that embodies its values of global accessibility, digital transformation, transparency, learning, and growth.

## Job Description

**Job Title: Product Designer**

### About Tiny

If you've ever written content on the web, chances are you've used a Tiny product. As part of the Tiugo Technologies portfolio of brands, we make TinyMCE and other great developer tools. Our team is dedicated to bringing content creation tools globally through open-source initiatives.

We're expanding our product family and are looking for a talented Product Designer to join our remote team of over 200 members worldwide, with significant presence in Australia, Poland, and the USA.

### About the Role

As a Product Designer for TinyMCE, you will have a direct impact on millions of users globally. In the TinyMCE product team, you will be part of the entire development journey, from identifying issues to defining and designing solutions. Your responsibilities include creating wireframes, prototypes, UI designs, and considering the broader experience for users, developers, and customers, such as editor behavior and best practices.

TinyMCE, our product, is a widely used component in customer's web apps. You will merge your ideas with those of other designers and developers responsible for integrating TinyMCE into their apps. The role involves functional, almost architectural design work, emphasizing research, user experience, usability, and empathy for both developers and end-users.

Familiarity with front-end web development is essential for discussing HTML and CSS, best practices, and accessibility standards with developers. Experience with design systems and componentized designs is beneficial.

Tiny is a fully remote organization with team members across multiple continents. Excellent communication skills are crucial, meaning you should be able to clearly express design briefs, documentation, and ideas in English, either in writing or video.

#### Key Responsibilities

- Translate customer feedback into actionable solutions.
- Collaborate with developers to implement solutions.
- Conduct industry and competitor research to identify opportunities, trends, and best practices.
- Assist the product team in communicating and visualizing upcoming features.
- Maintain and develop the TinyMCE design system in Figma.

#### Requirements

- 3-5 years of experience as a designer working with software development teams.
- Proficient in modern web development, able to effectively communicate technical details with developers.
- Familiarity with accessibility standards and their influence on design.
- Ability to work with at least a four-hour overlap with the CET timezone.
- Comfortable with English communication, both spoken and written.

### Why Join Us

- Flexible schedule and fully remote work.
- Professional development resources, including up to 5 days of additional leave and dedicated yearly funds.
- LinkedIn Learning access.
- Extra leave for Tiugo Day - our company’s foundation day, and your birthday.
- Access to a mental health and well-being platform.

## Interview Agenda

The interview process for the Product Designer role at Tiny will be conducted remotely and is structured as follows:

1. Introduction (5 minutes)
   - Welcome the candidate and provide an overview of Tiny and the TinyMCE product.
2. Candidate's Background (10 minutes)
   - Discuss the candidate's previous experience and how it relates to the Product Designer role.
3. Role-Specific Questions (30 minutes)
   - Delve into the candidate's competencies as outlined in the framework below.
4. Case Study / Role Play (20 minutes)
   - Present a case study related to TinyMCE and observe the candidate's approach to problem-solving.
5. Candidate Q&A (10 minutes)
   - Allow the candidate to ask questions about Tiny, the role, and the team.
6. Closing (5 minutes)
   - Thank the candidate for their time and explain the next steps in the hiring process.

## Competency Framework

The competency framework for the Product Designer role at Tiny is designed to assess a range of KSAOs that are critical for success. These competencies are aligned with Tiny's values of global accessibility, digital transformation, transparency, learning, and growth. The framework includes both technical and soft skills, ensuring that candidates are well-rounded and fit Tiny's remote-first culture.

1. **Design Proficiency**: Demonstrated ability to create wireframes, prototypes, and UI designs, with a strong understanding of design principles and best practices.
2. **Technical Communication**: Proficiency in discussing technical details related to HTML, CSS, and web development, and the ability to communicate these effectively with developers.
3. **User Empathy**: The capacity to understand and prioritize the needs of both developers and end-users, ensuring a seamless experience.
4. **Collaboration**: Ability to work effectively with a remote, cross-functional team, contributing to a shared vision and goals.
5. **Research and Analysis**: Skills in conducting industry and competitor research to inform design decisions and identify trends.
6. **Design Systems**: Experience with developing and maintaining design systems, preferably with knowledge of Figma.
7. **Accessibility Awareness**: Familiarity with accessibility standards and their application in design to ensure inclusivity.
8. **Time Zone Flexibility**: Willingness and ability to work with team members across different time zones, with a minimum four-hour overlap with the CET timezone.
9. **Language Proficiency**: Excellent English communication skills, both written and verbal, suitable for a fully remote work environment.

(Competencies are well-defined and cover the requirements mentioned in the job description. They are aligned with the company's values and the specific skills needed for the role. No changes are necessary.)

## Competency 1. **Design Proficiency**

Design proficiency is a core competency for the Product Designer role at Tiny, reflecting the company's commitment to delivering high-quality content creation tools. Candidates must demonstrate a strong portfolio of design work, showcasing their ability to create intuitive and visually appealing interfaces.

### Lead Question

**&ldquo;Can you walk us through your design process when approaching a new project, and how do you ensure that your designs meet both user needs and technical constraints?&rdquo;**

&nbsp;

&nbsp;

_What to look for: A structured design process, examples of balancing user needs with technical constraints, and a portfolio of relevant design work._

### Situational Scenario

**&ldquo;Imagine you are tasked with redesigning a key feature in TinyMCE that has received mixed feedback from users. How would you approach this challenge, and what steps would you take to gather user insights and iterate on your designs?&rdquo;**

&nbsp;

&nbsp;

_What to look for: A methodical approach to user feedback, problem-solving skills, and the ability to iterate on designs based on insights._

### Diving Deeper

- &ldquo;How do you stay updated with the latest design trends and technologies, and how do you apply them to your work?&rdquo;
- &ldquo;Can you provide an example of a design challenge you faced and how you overcame it?&rdquo;
- &ldquo;How do you measure the success of your designs, and what metrics do you use?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Lacks a clear design process and portfolio is not aligned with role requirements. | Shows basic design skills but lacks depth in process and understanding of user needs. | Demonstrates a good design process and understanding of user needs, with a solid portfolio. | Exhibits a strong design process, deep understanding of user needs, and a high-quality portfolio. | Exceptional design proficiency with an outstanding portfolio and a proven track record of innovative solutions. |

## Competency 2. **Technical Communication**

Technical communication is essential for the Product Designer role at Tiny, as it reflects the company's value of transparency and the need for clear communication in a remote work environment. Candidates must be able to discuss technical details with developers and ensure that design decisions are feasible and aligned with web development best practices.

### Lead Question

**&ldquo;How do you communicate your design decisions to a development team, and can you give an example of a time when you had to advocate for a design based on technical considerations?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Clear communication skills, examples of effective collaboration with developers, and the ability to advocate for design decisions._

### Situational Scenario

**&ldquo;You are working on a new feature for TinyMCE that requires complex front-end development. How would you ensure that your designs are implemented accurately and efficiently by the development team?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Strategies for clear and effective communication, understanding of technical constraints, and collaboration with developers._

### Diving Deeper

- &ldquo;Describe a situation where there was a technical limitation that impacted your design. How did you handle it?&rdquo;
- &ldquo;How do you ensure that your designs are technically feasible and maintainable in the long term?&rdquo;
- &ldquo;What tools or methods do you use to facilitate communication with remote development teams?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Struggles to communicate technical details and lacks examples of effective collaboration. | Can communicate some technical details but lacks confidence or experience in collaboration. | Communicates technical details effectively and has experience collaborating with developers. | Strong communicator with a track record of successful collaboration and technical advocacy. | Exceptional communicator with a deep understanding of technical considerations and a history of influencing successful outcomes. |

## Competency 3. **User Empathy**

User empathy is a key competency for the Product Designer role at Tiny, aligning with the company's value of global accessibility and the focus on creating tools that cater to a diverse user base. Candidates must demonstrate an ability to understand and prioritize the needs of both developers and end-users.

### Lead Question

**&ldquo;How do you ensure that your designs are user-centered, and can you share an example of how you incorporated user feedback into a design project?&rdquo;**

&nbsp;

&nbsp;

_What to look for: A user-centered design approach, examples of incorporating user feedback, and the ability to empathize with users._

### Situational Scenario

**&ldquo;A segment of TinyMCE users is struggling with a particular feature that is not intuitive. How would you go about redesigning this feature to improve the user experience?&rdquo;**

&nbsp;

&nbsp;

_What to look for: A thoughtful approach to user research, problem-solving skills, and the ability to design for user needs._

### Diving Deeper

- &ldquo;How do you balance the needs of different user groups when designing a product?&rdquo;
- &ldquo;Can you describe a time when you had to make a tough design decision based on user research findings?&rdquo;
- &ldquo;What methods do you use to gather and analyze user feedback?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Lacks a user-centered approach and struggles to incorporate user feedback into designs. | Shows some understanding of user needs but lacks depth in user research and feedback integration. | Good at incorporating user feedback and has a user-centered approach to design. | Strong user empathy with a track record of improving user experience based on feedback. | Exceptional user empathy with innovative solutions that significantly enhance user experience. |

## Competency 4. **Collaboration**

Collaboration is crucial for the Product Designer role at Tiny, reflecting the company's remote-first culture and the need for effective teamwork across continents. Candidates must demonstrate their ability to work well with others, contribute to a shared vision, and communicate effectively in a remote setting.

### Lead Question

**&ldquo;Can you provide an example of a successful project you worked on with a remote team, and what were the key factors that contributed to its success?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Examples of successful remote collaboration, communication skills, and the ability to contribute to team goals._

### Situational Scenario

**&ldquo;You are part of a remote team working on a tight deadline for a new TinyMCE feature. How would you ensure effective collaboration and communication among team members to meet the deadline?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Strategies for effective remote collaboration, time management skills, and the ability to work under pressure._

### Diving Deeper

- &ldquo;How do you build and maintain relationships with remote team members?&rdquo;
- &ldquo;What tools and practices do you use to stay aligned with your team when working remotely?&rdquo;
- &ldquo;Describe a challenge you faced while collaborating remotely and how you overcame it.&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Struggles with remote collaboration and lacks examples of contributing to team success. | Some experience with remote collaboration but lacks effective communication strategies. | Good remote collaborator with solid communication skills and a team-oriented mindset. | Strong collaborator with a proven track record of remote teamwork and effective communication. | Exceptional collaborator with outstanding remote teamwork skills and a history of driving team success. |

## Competency 5. **Research and Analysis**

Research and analysis are important competencies for the Product Designer role at Tiny, aligning with the company's focus on innovation and staying ahead of industry trends. Candidates must demonstrate their ability to conduct thorough research and use their findings to inform design decisions.

### Lead Question

**&ldquo;How do you stay informed about industry trends and competitor products, and how does this research influence your design work?&rdquo;**

&nbsp;

&nbsp;

_What to look for: A systematic approach to research, the ability to analyze trends, and the application of research findings to design._

### Situational Scenario

**&ldquo;You have been asked to identify new opportunities for TinyMCE based on industry trends and competitor analysis. How would you approach this task, and what would be your process for sharing your findings with the team?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Research skills, analytical thinking, and the ability to communicate insights effectively._

### Diving Deeper

- &ldquo;Can you describe a time when your research led to a significant change in a design project?&rdquo;
- &ldquo;How do you validate the insights you gain from your research?&rdquo;
- &ldquo;What tools or resources do you use for conducting industry and competitor research?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Lacks a structured approach to research and struggles to apply findings to design work. | Conducts basic research but lacks depth in analysis and application to design. | Good research skills with the ability to apply findings to improve design outcomes. | Strong research and analytical skills with a history of using insights to drive design innovation. | Exceptional research and analysis capabilities with a proven impact on product design and strategy. |

## Competency 6. **Design Systems**

Experience with design systems is a valuable competency for the Product Designer role at Tiny, reflecting the company's emphasis on consistency and scalability in design. Candidates must demonstrate their experience with developing and maintaining design systems, preferably with knowledge of Figma.

### Lead Question

**&ldquo;Can you discuss your experience with design systems and how you have contributed to their development or maintenance?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Experience with design systems, contributions to their development, and familiarity with tools like Figma._

### Situational Scenario

**&ldquo;The TinyMCE design system needs to be updated to accommodate new features and ensure consistency across the product. How would you approach this task, and what steps would you take to ensure the design system remains scalable and easy to use for the development team?&rdquo;**

&nbsp;

&nbsp;

_What to look for: A methodical approach to updating design systems, understanding of scalability, and the ability to work with development teams._

### Diving Deeper

- &ldquo;How do you ensure that a design system is adopted and used effectively by a development team?&rdquo;
- &ldquo;Can you provide an example of a challenge you faced with a design system and how you resolved it?&rdquo;
- &ldquo;What role do you believe design systems play in the overall user experience of a product?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Limited experience with design systems and lacks understanding of their importance. | Some experience with design systems but lacks contributions to their development or maintenance. | Good experience with design systems and has contributed to their development. | Strong experience with design systems, including significant contributions to their scalability and maintenance. | Exceptional experience with design systems, with a proven track record of innovation and effective implementation. |

## Competency 7. **Accessibility Awareness**

Accessibility awareness is a critical competency for the Product Designer role at Tiny, aligning with the company's value of inclusivity and the need to create products that are accessible to all users. Candidates must demonstrate their familiarity with accessibility standards and their application in design.

### Lead Question

**&ldquo;How do you incorporate accessibility standards into your design process, and can you give an example of a project where you improved accessibility?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Understanding of accessibility standards, examples of incorporating accessibility into designs, and a commitment to inclusivity._

### Situational Scenario

**&ldquo;You are designing a new feature for TinyMCE that must be accessible to users with disabilities. What steps would you take to ensure that the design meets accessibility standards?&rdquo;**

&nbsp;

&nbsp;

_What to look for: A structured approach to accessibility, knowledge of standards, and the ability to design for inclusivity._

### Diving Deeper

- &ldquo;What resources do you use to stay informed about accessibility standards and best practices?&rdquo;
- &ldquo;How do you test your designs for accessibility compliance?&rdquo;
- &ldquo;Can you describe a situation where you had to balance design aesthetics with accessibility requirements?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Lacks understanding of accessibility standards and has not applied them in design projects. | Some awareness of accessibility but limited application in design work. | Good understanding of accessibility standards and has applied them in some projects. | Strong commitment to accessibility with a track record of designing inclusive products. | Exceptional knowledge of accessibility with a history of significantly improving product accessibility. |

## Competency 8. **Time Zone Flexibility**

Time zone flexibility is important for the Product Designer role at Tiny, reflecting the company's remote-first culture and the need for collaboration across multiple continents. Candidates must demonstrate their willingness and ability to work with team members in different time zones.

### Lead Question

**&ldquo;How do you manage your work schedule to collaborate effectively with team members in different time zones, and can you provide an example of how you've done this in the past?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Time management skills, flexibility, and examples of successful collaboration across time zones._

### Situational Scenario

**&ldquo;You are working with developers in Poland and Australia on a critical project for TinyMCE. How would you ensure that you are available for collaboration and communication despite the time zone differences?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Strategies for managing time zone differences, communication skills, and the ability to adapt to various schedules._

### Diving Deeper

- &ldquo;What tools or methods do you use to keep track of different time zones when scheduling meetings or deadlines?&rdquo;
- &ldquo;How do you handle situations where time zone differences lead to delays or miscommunications?&rdquo;
- &ldquo;Can you discuss a time when you had to adjust your work hours to meet the needs of a project or team?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Struggles with time zone management and lacks experience working with remote teams. | Some experience with time zone management but lacks flexibility or effective strategies. | Good at managing time zone differences and has experience working with remote teams. | Strong time management skills and a proven ability to adapt to different time zones. | Exceptional flexibility and a history of seamless collaboration across multiple time zones. |

## Competency 9. **Language Proficiency**

Language proficiency, specifically in English, is crucial for the Product Designer role at Tiny, as it ensures clear communication in a fully remote work environment. Candidates must demonstrate excellent spoken and written English skills.

### Lead Question

**&ldquo;Can you provide examples of design documentation or briefs you have written in English, and how do you ensure clarity in your written communication?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Strong written communication skills, clarity in documentation, and examples of written work._

### Situational Scenario

**&ldquo;You need to present a design proposal to the TinyMCE team via video call. How would you prepare for this presentation to ensure that your ideas are clearly communicated and understood by the team?&rdquo;**

&nbsp;

&nbsp;

_What to look for: Preparation strategies, verbal communication skills, and the ability to convey ideas effectively._

### Diving Deeper

- &ldquo;How do you handle language barriers or misunderstandings when communicating with team members whose first language is not English?&rdquo;
- &ldquo;What practices do you follow to improve your English communication skills?&rdquo;
- &ldquo;Can you discuss a time when your communication skills helped to resolve a conflict or misunderstanding within a team?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Struggles with English communication and lacks clear written or verbal skills. | Basic English communication skills but lacks clarity or confidence in expression. | Good English communication skills with clear written and verbal expression. | Strong English communication skills with a track record of effective documentation and presentations. | Exceptional English proficiency with outstanding communication skills and a history of successful team collaboration. |

## Case Study / Role Play

To assess the practical application of skills, candidates will be presented with a case study that reflects a real-world scenario they might encounter at Tiny. The case study will involve redesigning a feature of TinyMCE based on user feedback and technical constraints. Candidates will be asked to create wireframes, prototypes, and UI designs, considering the broader experience for users, developers, and customers.

The role play will simulate collaboration with developers and other team members, requiring the candidate to communicate their design decisions and incorporate feedback. This exercise will demonstrate the candidate's design proficiency, technical communication, user empathy, and ability to work within a remote team.

The case study will also include considerations for accessibility and the maintenance of the TinyMCE design system, allowing the candidate to showcase their knowledge and experience in these areas. The outcome of the case study will provide valuable insights into the candidate's problem-solving abilities and their fit within Tiny's culture and values.

## Candidate Q&A

At the end of the interview, candidates will have the opportunity to ask questions about Tiny, the Product Designer role, and the team they will be working with. This section of the interview is designed to gauge the candidate's interest in the company and the role, as well as their alignment with Tiny's values and vision.

Candidates are encouraged to inquire about Tiny's remote work culture, opportunities for professional development, and the company's approach to content creation and digital transformation. This Q&A session will also allow the interviewers to clarify any points and provide additional information that may be important to the candidate.

**Q. What does a typical day look like for a Product Designer at Tiny?**
A. Product Designers at Tiny are involved in various activities, from conducting research and creating designs to collaborating with developers and stakeholders. They work in a fully remote environment, which offers flexibility in managing their schedule.

**Q. How does Tiny support professional development for its team members?**
A. Tiny provides professional development resources, including additional leave and dedicated yearly funds, as well as access to LinkedIn Learning and other platforms.

**Q. Can you tell me more about the TinyMCE design system and how it's maintained?**
A. The TinyMCE design system is maintained in Figma and is a collaborative effort involving designers and developers. It ensures consistency and scalability across the product.

**Q. How does Tiny handle feedback and incorporate it into the product development process?**
A. Tiny values feedback from users, developers, and team members. It has mechanisms for collecting feedback and processes for integrating insights into product development.

**Q. What opportunities are there for collaboration and learning from other team members?**
A. Tiny's remote-first culture fosters collaboration through various online tools and platforms. Team members have opportunities to learn from each other and contribute to shared goals.

**Q. How does Tiny ensure that its products remain accessible and user-friendly?**
A. Tiny prioritizes accessibility and user experience in its design process. The team stays informed about accessibility standards and best practices to ensure that products are inclusive and easy to use.

## Candidate Evaluation

Complete the following scorecard for the Product Designer role at Tiny. Refer to the evaluation guides for each competency to determine the candidate's score.

| Competency | Rating (1-5) | Interviewer Comments |
| --- | --- | --- |
| Design Proficiency |   |   |
| Technical Communication |   |   |
| User Empathy |   |   |
| Collaboration |   |   |
| Research and Analysis |   |   |
| Design Systems |   |   |
| Accessibility Awareness |   |   |
| Time Zone Flexibility |   |   |
| Language Proficiency |   |   |
| &nbsp; | | |
| **Total** |   |   |

